
# Práctico — Programación Dinámica: Fibonacci (sin/ con memoización) y Mochila 0/1

**Objetivos**
- Contrastar *recursión ingenua* vs **memoización** en Fibonacci (tiempo, llamadas).
- Implementar **Mochila 0/1** con **tabulación** y **reconstrucción** sobre un **ejemplo generado aleatoriamente**.
- Interpretar resultados y complejidades.


In [1]:

import math, random, time
from functools import lru_cache
random.seed(42)
print("Semilla aleatoria:", 42)


Semilla aleatoria: 42


## 1) Fibonacci — recursión ingenua (sin memoización)

In [12]:

def fib_rec(n):
    if n <= 1:
        return n
    return fib_rec(n-1) + fib_rec(n-2)

n_small = 20
t0 = time.perf_counter()
val_small = fib_rec(n_small)
t1 = time.perf_counter()

print(f"fib_rec({n_small}) = {val_small}")
print(f"Tiempo (seg): {t1 - t0:.4f}")


fib_rec(20) = 6765
Tiempo (seg): 0.0011


## 2) Fibonacci — memoización (top-down)

In [13]:

def fib_memo(n, mem=None):
    """
    Calcula F(n) con memoización explícita.
    - n: entero >= 0
    - mem: diccionario para cachear resultados {k: F(k)}
    """   
    # TO DO: Completar el código de fibonnaci con memoización explícita, siguiendo la plantilla vista en clase.
    # Meoización explícita implica usar un diccionario para guardar los resultados ya calculados,
    # de esta forma evitamos cálculos repetidos. 
    if mem is None: 
        mem = {}
    if n in mem:
        return mem[n] # Si estado en mem: retornar mem[estado]
    if n <= 1:
        return n # Si estado es caso_base: retornar valor_base
    
    mem[n] = fib_memo(n-1, mem) + fib_memo(n-2, mem) # mejor <- combinar (F(subestado1), F(subestado2), ...)
    return mem[n] # mem[estado] <- mejor, y retornarlo

from functools import lru_cache
@lru_cache(maxsize=None)
def fib_memo_lru(n):
    if n <= 1:
        return n
    return fib_memo_lru(n-1) + fib_memo_lru(n-2)

n_big = 2000
mem = {}
t0 = time.perf_counter()
val = fib_memo(n_big, mem)
t1 = time.perf_counter()

print(f"fib_memo_dict({n_big}) = {val}")
print(f"Tiempo (seg): {t1 - t0:.6f}")
print(f"Llamadas únicas computadas: {len(mem)} (0..{max(mem.keys())})")

fib_memo_dict(2000) = 4224696333392304878706725602341482782579852840250681098010280137314308584370130707224123599639141511088446087538909603607640194711643596029271983312598737326253555802606991585915229492453904998722256795316982874482472992263901833716778060607011615497886719879858311468870876264597369086722884023654422295243347964480139515349562972087652656069529806499841977448720155612802665404554171717881930324025204312082516817125
Tiempo (seg): 0.002027
Llamadas únicas computadas: 1999 (0..2000)



**Comentarios rápidos:**  
- `fib_rec` crece **exponencial** en tiempo (≈ \(\varphi^n\)).  
- `fib_memo` es **O(n)** en tiempo y **O(n)** en memoria.  
- Para `n` muy grande, preferir *tabulación* iterativa (O(1) espacio) o *fast doubling* (O(log n)).


## 3) Mochila 0/1 — Tabulación + Reconstrucción (instancia aleatoria)

In [5]:

def generar_instancia_mochila(n_items=10, wmin=1, wmax=20, vmin=10, vmax=100, capacidad_ratio=0.4):
    pesos = [random.randint(wmin, wmax) for _ in range(n_items)]
    valores = [random.randint(vmin, vmax) for _ in range(n_items)]
    cap = max(1, int(sum(pesos) * capacidad_ratio))
    return pesos, valores, cap

pesos, valores, CAP = generar_instancia_mochila(n_items=10)
n = len(pesos)

print("Instancia generada:")
for i, (w, v) in enumerate(zip(pesos, valores), start=1):
    print(f"  Item {i:02d}: peso={w:2d}, valor={v:3d}")
print("Capacidad:", CAP)


Instancia generada:
  Item 01: peso= 4, valor= 64
  Item 02: peso= 1, valor= 14
  Item 03: peso= 9, valor= 13
  Item 04: peso= 8, valor= 21
  Item 05: peso= 8, valor= 37
  Item 06: peso= 5, valor= 39
  Item 07: peso= 4, valor= 74
  Item 08: peso=18, valor= 87
  Item 09: peso= 3, valor= 13
  Item 10: peso=19, valor= 81
Capacidad: 31


In [ ]:

def mochila_01_tab(pesos, valores, CAP):
    """
    Mochila 0/1 con Programación Dinámica (tabulación, bottom-up).

    Parámetros:
      - pesos:   lista de enteros > 0 con los pesos de cada ítem
      - valores: lista de enteros >= 0 con los valores de cada ítem
      - CAP:     capacidad máxima (entero >= 0)

    Retorna:
      - mejor_valor:  valor máximo alcanzable con capacidad CAP
      - items_tomados: lista de índices (0-based) de los ítems seleccionados
      - dp:           tabla DP de tamaño (n+1) x (CAP+1) con los valores óptimos
                      para subproblemas (primeras i filas, capacidad w)
    Idea:
      dp[i][w] = mejor valor usando SOLO los primeros i ítems y una capacidad w.
      Transición:
        - No tomar i:           dp[i-1][w]
        - Tomar i (si cabe):   valores[i-1] + dp[i-1][w - pesos[i-1]]
      Tomamos el máximo de ambas opciones.
    """
    n = len(pesos)

    # 1) Crear tabla DP con ceros
    dp = [[0] * (CAP + 1) for _ in range(n + 1)]

    # 2) Llenado de la tabla (bottom-up)
    for i in range(1, n + 1):
        peso_i = pesos[i - 1]
        valor_i = valores[i - 1]
        for w in range(CAP + 1):
            mejor = dp[i - 1][w] # Opción 1: NO tomar el ítem i-1
            if peso_i <= w:      # Opción 2: tomar el ítem i-1 si cabe
                mejor = max(mejor, valor_i + dp[i - 1][w - peso_i])
            dp[i][w] = mejor

    # 3) Reconstrucción de la solución óptima
    w = CAP
    items_tomados = []
    for i in range(n, 0, -1):
        if dp[i][w] != dp[i - 1][w]:
            items_tomados.append(i - 1)
            w -= pesos[i - 1]
            if w == 0:
                break

    items_tomados.reverse()
    return dp[n][CAP], items_tomados, dp


# ==========================
# Ejecución/medición ejemplo
# (Asume que ya existen: pesos, valores, CAP)
# ==========================
t0 = time.perf_counter()
mejor_valor, items_tomados, dp = mochila_01_tab(pesos, valores, CAP)
t1 = time.perf_counter()

# Resultados principales
print(f"Mejor valor total: {mejor_valor}")
print(f"Ítems tomados (índices 0-based): {items_tomados}")

# Verificación y métricas útiles
if items_tomados:
    total_peso = sum(pesos[i] for i in items_tomados)
    total_valor = sum(valores[i] for i in items_tomados)
    print(f"Peso total: {total_peso} de {CAP}")
    print(f"Valor total (verificación): {total_valor}")

print(f"Tiempo DP (seg): {t1 - t0:.6f}")

Mejor valor total: 264
Ítems tomados (índices 0-based): [0, 5, 6, 7]
Peso total: 31 de 31
Valor total (verificación): 264
Tiempo DP (seg): 0.000183



**Notas:**  
- Estado: `dp[i][W]` = mejor valor con los primeros `i` ítems y capacidad `W`.  
- Transición: `max( dp[i-1][W], valor_i + dp[i-1][W - peso_i] )` cuando `peso_i ≤ W`.  
- Complejidad: **O(n · CAP)** tiempo y **O(n · CAP)** memoria.  
- *Extra (no implementado aquí)*: versión **1D** usando `dp[W]` y recorriendo `W` **de mayor a menor**.


### (Opcional) Otra instancia: cambie la semilla y re-ejecute
### Imprima el resultado de Mochila 0/1 en un una tabla que refleje el proceso de ejecución del algoritmo

In [21]:

random.seed(123)
pesos, valores, CAP = generar_instancia_mochila(n_items=12)
mejor_valor, items_tomados, dp = mochila_01_tab(pesos, valores, CAP)
print("Capacidad:", CAP, "| Mejor valor:", mejor_valor, "| Ítems:", items_tomados)

random.seed(598)
pesos, valores, CAP = generar_instancia_mochila(n_items=12)
mejor_valor, items_tomados, dp = mochila_01_tab(pesos, valores, CAP)
print("Capacidad:", CAP, "| Mejor valor:", mejor_valor, "| Ítems:", items_tomados)

random.seed(87)
pesos, valores, CAP = generar_instancia_mochila(n_items=12)
mejor_valor, items_tomados, dp = mochila_01_tab(pesos, valores, CAP)
print("Capacidad:", CAP, "| Mejor valor:", mejor_valor, "| Ítems:", items_tomados)

random.seed(95)
pesos, valores, CAP = generar_instancia_mochila(n_items=12)
mejor_valor, items_tomados, dp = mochila_01_tab(pesos, valores, CAP)
print("Capacidad:", CAP, "| Mejor valor:", mejor_valor, "| Ítems:", items_tomados)


Capacidad: 45 | Mejor valor: 393 | Ítems: [0, 2, 3, 4, 5, 6, 10]
Capacidad: 51 | Mejor valor: 482 | Ítems: [0, 1, 3, 4, 8, 11]
Capacidad: 43 | Mejor valor: 488 | Ítems: [0, 3, 4, 5, 8, 10, 11]
Capacidad: 48 | Mejor valor: 536 | Ítems: [1, 3, 5, 6, 7, 9, 10, 11]
